# STRUMOK and SNOW 2.0 implementations

In [117]:
from typing import Union
from qiskit import Aer, QuantumCircuit, QuantumRegister
from qiskit.circuit import Qubit, Instruction

In [118]:
simulator = Aer.get_backend('qasm_simulator')

## Utility Functions

In [119]:
"""Load byte s-box"""
def load_function_table(name: str) -> list:
    file = open(name, 'r')
    table = file.read().replace('\n', ' ').split(' ')
    file.close()
    for x in range(len(table)):
        table[x] = int(table[x], 16)
    return table


In [120]:
"""
Fast Mobius Transform of the function with input
and output of length 8. The function must be represented
as table indexed by function input.
"""
def fmt(table: list) -> list:
    anf = table.copy()
    for i in range(8):
        bit_flip = 1 << i
        for j in range(1 << 8):
            if (j >> i) & 0x1 != 0:
                anf[j] = anf[j] ^ anf[j ^ bit_flip]
    return anf


In [121]:
"""
Return list of the bits that are set in the vector
that is represented as int. Considered only eight
least significant bits.
"""
def vector_to_indices(vector: int) -> list:
    indexes = []
    for i in range(8):
        if (vector >> i) & 0x1 == 0x1:
            indexes.append(i)
    return indexes

In [122]:
"""
Explode input registers return a flat
structure (list of qubits preserving the
order).

Supported input types: Qubit, QuantumRegister or
list of Qubits or QuantumRegisters
"""
def explode(*registers) -> list:
    exploded = []
    for reg in registers:
        if isinstance(reg, list):
            exploded.extend(explode(*reg))
        elif isinstance(reg, QuantumRegister):
            for index in range(reg.size):
                exploded.append(reg[index])
        elif isinstance(reg, Qubit):
            exploded.append(reg)
        else:
            raise Exception("Unexpected quantum storage type")
    return exploded

In [123]:
def generate_byte_field(reduction_poly: int) -> list:
    field = [0x1]
    elem = 0x1
    for _ in range(1, 1 << 8):
        elem = elem << 1
        if elem >> 8 == 0x1:
            elem = (elem ^ reduction_poly) & 0xFF
        field.append(elem)
    return field

## Base Circuits

In [124]:
"""
Qubit-wise XOR.

length - length of the quantum registers

x, y - input quantum registers
out - output quantum register

Bra-ket representation: |x>|y>|out> -> |x>|y>|out ^ x ^ y>
"""
def xor_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')

    xor = QuantumCircuit(x, y, out, name=format("xor_%s", str(length)))

    for i in range(0, length):
        xor.cx(x[i], out[i])
        xor.cx(y[i], out[i])

    return xor

In [125]:
"""
Qubit-wise inplace XOR (Multiqubit CNOT).

length - length of the quantum registers

ctr - input quantum registers
trg - output quantum register

Bra-key representation: |ctr>|trg> -> |ctr>|trg ^ ctr>
"""
def inplace_xor_circuit(length: int):
    ctr = QuantumRegister(length, 'ctr')
    trg = QuantumRegister(length, 'trg')

    circuit = QuantumCircuit(ctr, trg, name='inplace_xor_' + str(length))

    for i in range(length):
        circuit.cx(ctr[i], trg[i])

    return circuit

In [126]:
"""
Inverts all qubits in the quantum register.

length - length of the quantum registers

Bra-ket representation: |x> -> |~x>
"""
def not_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')

    circuit = QuantumCircuit(x, name='not_' + str(length))

    for i in range(length):
        circuit.x(i)

    return circuit

In [127]:
"""
Swaps two quantum registers.

length - length of the quantum registers

Bra-ket representation: |x>|y> -> |y>|x>
"""
def swap_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')

    circuit = QuantumCircuit(x, y, name='swap_' + str(length))

    for i in range(length):
        circuit.swap(x[i], y[i])

    return circuit

In [128]:
"""
Reset quantum register. It is used to reset ancillary qubits.

length - length of the quantum register

Bra-ket representation: |x> -> |0>
"""
def reset_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')

    circuit = QuantumCircuit(x, name='reset_' + str(length))

    for i in range(length):
        circuit.reset(x[i])

    return circuit

In [129]:
"""
Base full adder quantum circuit.

Qubits: 0 - x, 1 - y, 2 - carry, 3 - out

Bra-ket representation:
|x>|y>|carry>|out> -> |x>|y>|(x + y + carry) / 2>|out ^ ((x + y + carry) % 2)>
"""
def base_full_adder() -> QuantumCircuit:
     full_adder = QuantumCircuit(4, name='full_adder')
     full_adder.ccx(0, 1, 3)
     full_adder.cx(0, 1)
     full_adder.ccx(1, 2, 3)
     full_adder.cx(1, 2)
     full_adder.cx(0, 1)
     full_adder.swap(2, 3)
     return full_adder

In [130]:
"""
Full adder quantum circuit.

Expected carry qubit to be in |0> state at the beginning.
"""
def full_adder_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    if length == 1:
        return base_full_adder()

    base_adder = base_full_adder().to_instruction()

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')
    carry = QuantumRegister(1, 'carry')

    adder = QuantumCircuit(x, y, out, carry, name=format("add_%s", str(length)))

    for i in range(0, length):
        adder.append(base_adder, [x[i], y[i], carry[0], out[i]])

    return adder

In [131]:
"""
S-box quantum circuit created from ANF. Accepted only
function with input and output of eight length.

anf - table of ANF coefficients index by input variables
box_name - s-box string name

Quantum registers: x, out
"""
def s_box_circuit(anf: list, box_name: str) -> QuantumCircuit:
    assert len(anf) == (1 << 8)

    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name="s_box_" + box_name)

    for i in range(1 << 8):
        if anf[i] != 0:
            if i == 0:
                for index in vector_to_indices(anf[i]):
                    circuit.x(out[index])
            else:
                for index in vector_to_indices(anf[i]):
                    circuit.mcx(x[vector_to_indices(i)], out[index])

    return circuit

In [132]:
"""
General quantum circuit for multiplication
in field GF(2^8).

Parameters:
reduction_circuit - quantum circuit of inplace reduction
of high degree polynomial components

Registers: x, y, out - of length 8
"""
def mult_f8_circuit(reduction_circuit: Union[QuantumCircuit, Instruction]) -> QuantumCircuit:
    x = QuantumRegister(8, 'x')
    y = QuantumRegister(8, 'y')
    out = QuantumRegister(8, 'out')

    mult = QuantumCircuit(x, y, out, name='mult_f8')

    for i in range(1, 8):
        for j in range(8 - i):
            mult.ccx(x[i + j], y[7 - j], out[i - 1])

    mult.append(reduction_circuit, qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            mult.ccx(x[j], y[i - j], out[i])

    return mult

In [133]:
"""
Specific quantum circuit for multiplication
in field GF(2^8) by constant field element.

Parameters:
reduction_circuit - quantum circuit of inplace reduction
of high degree polynomial components
constant - element of GF(2^8)
name - element name

Registers: x, out - of length 8
"""
def mult_const_f8_circuit(reduction_circuit: Union[QuantumCircuit, Instruction],
                          constant: int,
                          name=None) -> QuantumCircuit:

    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    const_vector = []
    for i in range(8):
        const_vector.append((constant >> i) & 0x1)

    if name is None:
        name = hex(constant)

    mult = QuantumCircuit(x, out, name='mult_f8_' + name)

    for i in range(1, 8):
        for j in range(8 - i):
            if const_vector[i + j] == 0x1:
                mult.cx(x[7 - j], out[i - 1])

    mult.append(reduction_circuit, qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            if const_vector[j] == 0x1:
                mult.cx(x[i - j], out[i])

    return mult

## STRUMOK circuits

In [134]:
"""
Quantum circuit for performing in place
reduction of high polynomial components
during multiplication in filed GF(2^8).

Input-output size: 8
"""
def strumok_qt_reduction_circuit() -> QuantumCircuit:
    qt = QuantumCircuit(8, name="qt_reduction")

    qt.cx(4, 0)
    qt.cx(5, 0)
    qt.cx(6, 0)
    qt.cx(5, 1)
    qt.cx(6, 1)
    qt.cx(6, 2)

    qt.barrier()

    qt.cx(5, 7)
    qt.cx(4, 7)
    qt.cx(4, 6)
    qt.cx(3, 7)
    qt.cx(3, 6)
    qt.cx(3, 5)
    qt.cx(2, 6)
    qt.cx(2, 5)
    qt.cx(2, 4)
    qt.cx(1, 5)
    qt.cx(1, 4)
    qt.cx(1, 3)
    qt.cx(0, 4)
    qt.cx(0, 3)
    qt.cx(0, 2)

    return qt

In [135]:
# Generating STRUMOK field GF(2^8) index by reduction
# polynomial root

strumok_reduction_poly = 0b00011101
strumok_field = generate_byte_field(strumok_reduction_poly)

In [136]:
def strumok_mult_alpha_circuit() -> QuantumCircuit:
    str_qt_reduction = strumok_qt_reduction_circuit().to_instruction()
    add_f8 = inplace_xor_circuit(8).to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_alpha_f64')

    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[2  ], 'b^2'  ), qargs=explode(xs[7], outs[0]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[70 ], 'b^70' ), qargs=explode(xs[7], outs[3]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[224], 'b^224'), qargs=explode(xs[7], outs[4]))
    circuit.append(add_f8, qargs=explode(outs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[166], 'b^166'), qargs=explode(xs[7], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i], outs[i + 1]))

    return circuit

In [137]:
def strumok_mult_inv_alpha_circuit() -> QuantumCircuit:
    str_qt_reduction = strumok_qt_reduction_circuit()
    add_f8 = inplace_xor_circuit(8).to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_inv_alpha_f64')

    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[68 ], 'b^68' ), qargs=explode(xs[0], outs[2]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[222], 'b^222'), qargs=explode(xs[0], outs[3]))
    circuit.append(add_f8, qargs=explode(xs[0], outs[4]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[164], 'b^164'), qargs=explode(xs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(str_qt_reduction, strumok_field[168], 'b^168'), qargs=explode(xs[0], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i + 1], outs[i]))

    return circuit

In [138]:
L = [[0x01, 0x01, 0x05, 0x01, 0x08, 0x6, 0x07, 0x04],
    [0x04, 0x01, 0x01, 0x05, 0x01, 0x08, 0x06, 0x07],
    [0x07, 0x04, 0x01, 0x01, 0x05, 0x01, 0x08, 0x06],
    [0x06, 0x07, 0x04, 0x01, 0x01, 0x05, 0x01, 0x08],
    [0x08, 0x06, 0x07, 0x04, 0x01, 0x01, 0x05, 0x01],
    [0x01, 0x08, 0x06, 0x07, 0x04, 0x01, 0x01, 0x05],
    [0x05, 0x01, 0x08, 0x06, 0x07, 0x04, 0x01, 0x01],
    [0x01, 0x05, 0x01, 0x08, 0x06, 0x07, 0x04, 0x01]]

In [139]:
def strumok_substitution_circuit() -> QuantumCircuit:
    reset_8 = reset_circuit(8)

    x = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    out = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]
    tmp = QuantumRegister(8, 'tmp')

    regs = []
    regs.extend(x)
    regs.extend(out)
    regs.append(tmp)

    circuit = QuantumCircuit(*regs, name='strumok_tau')

    for i in range(8):
        index = i % 4
        s_box = s_box_circuit(fmt(load_function_table('./sbox/strumok/pi_' + str(index) + '.txt')), str(index))

        circuit.append(reset_8, qargs=explode(tmp))
        circuit.append(s_box, qargs=explode(x[i], tmp))

        for j in range(8):
            circuit.append(mult_const_f8_circuit(strumok_qt_reduction_circuit(), L[j][i]), qargs=explode(tmp, out[j]))

    return circuit



In [140]:
def strumok_fsm_circuit() -> QuantumCircuit:
    x = QuantumRegister(64, 'x')
    y = QuantumRegister(64, 'y')
    z = QuantumRegister(64, 'z')
    out = QuantumRegister(64, 'out')
    carry = QuantumRegister(1, 'carry')

    circuit = QuantumCircuit(x, y, z, out, carry, name='strumok_fsm')
    circuit.append(full_adder_circuit(64), qargs=explode(x, y, out, carry))
    circuit.append(inplace_xor_circuit(64), qargs=explode(z, out))

    return circuit

In [141]:
"""
Next function of STRUMOK-256 stream cipher

Parameters:
init_mode - boolean flag used to specify NEXT function mode

Quantum Register:
s - register that stores LFSR state (length 16 * 64 qubits)
r_1 - register of FSM (length 64 qubits)
r_2 - register of FSM (length 64 qubits)
tmp - temporary register of length 64 qubits
carry - temporary qubit for storing carry bit during addition
"""
def strumok_next_circuit(init_mode=False) -> QuantumCircuit:
    swap_64 = swap_circuit(64).to_instruction()
    reset_64 = reset_circuit(64).to_instruction()
    iadd_f64 = inplace_xor_circuit(64).to_instruction()

    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    tmp = QuantumRegister(64, 'tmp')
    carry = QuantumRegister(1, 'carry')

    # Circuit Input
    regs = []
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(tmp)
    regs.append(carry)

    circuit = QuantumCircuit(*regs, name='strumok_next')

    circuit.append(strumok_substitution_circuit(), qargs=explode(r_1, r_2, tmp[0:8]))
    circuit.append(reset_64, qargs=explode(r_1))
    circuit.append(full_adder_circuit(64), qargs=explode(r_2, s[13], r_1, carry[0]))

    circuit.append(reset_64, qargs=explode(tmp))
    circuit.append(iadd_f64, qargs=explode(s[13], tmp))
    circuit.append(strumok_mult_alpha_circuit(), qargs=explode(s[0], tmp))
    circuit.append(strumok_mult_inv_alpha_circuit(), qargs=explode(s[11], tmp))

    if init_mode:
        circuit.reset(carry[0])
        circuit.append(strumok_fsm_circuit(), qargs=explode(s[15], r_1, r_2, tmp, carry[0]))

    # After this rotation s[i] -> s[i - 1], where s[0] -> s[15]
    for j in range(14, -1, -1):
        circuit.append(swap_64, explode(s[15], s[j]))

    circuit.append(reset_64, qargs=explode(s[15]))
    circuit.append(iadd_f64, qargs=explode(tmp, s[15]))

    return circuit

In [142]:
"""
Init function of STRUMOK-256 stream cipher

Quantum Register:
key - register that stores cipher key of length 256 qubit
iv - register that stores cipher iv of length 256
s - register that stores LFSR state (length 16 * 64 qubits)
r_1 - register of FSM (length 64 qubits)
r_2 - register of FSM (length 64 qubits)
tmp - temporary register of length 64 qubits
carry - temporary qubit for storing carry bit during addition
"""
def strumok_init_circuit() -> QuantumCircuit:
    not_64 = not_circuit(64).to_instruction()
    cnot_64 = inplace_xor_circuit(64).to_instruction()
    reset_64 = reset_circuit(64).to_instruction()

    strumok_next = strumok_next_circuit(True).to_instruction()

    # Cipher Input
    key = [QuantumRegister(64, 'k_' + str(i)) for i in range(4)]
    iv = [QuantumRegister(64, 'iv_' + str(i)) for i in range(4)]

    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    tmp = QuantumRegister(64, 'tmp')
    carry = QuantumRegister(1, 'carry')

    # Circuit Input
    regs = []
    regs.extend(key)
    regs.extend(iv)
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(tmp)
    regs.append(carry)

    circuit = QuantumCircuit(*regs, name='strumok_init')

    circuit.append(cnot_64, qargs=explode(key[0], s[15]))
    circuit.append(not_64, qargs=explode(s[15]))
    circuit.append(cnot_64, qargs=explode(key[1], s[14]))
    circuit.append(cnot_64, qargs=explode(key[2], s[13]))
    circuit.append(not_64, qargs=explode(s[13]))
    circuit.append(cnot_64, qargs=explode(key[3], s[12]))
    circuit.append(cnot_64, qargs=explode(key[0], s[11]))
    circuit.append(cnot_64, qargs=explode(key[1], s[10]))
    circuit.append(not_64, qargs=explode(s[10]))
    circuit.append(cnot_64, qargs=explode(key[2], s[9]))
    circuit.append(cnot_64, qargs=explode(key[3], s[8]))
    circuit.append(cnot_64, qargs=explode(key[0], s[7]))
    circuit.append(not_64, qargs=explode(s[7]))
    circuit.append(cnot_64, qargs=explode(key[1], s[6]))
    circuit.append(not_64, qargs=explode(s[6]))
    circuit.append(cnot_64, qargs=explode(key[2], s[5]))
    circuit.append(cnot_64, qargs=explode(iv[3], s[5]))
    circuit.append(cnot_64, qargs=explode(key[3], s[4]))
    circuit.append(cnot_64, qargs=explode(key[0], s[3]))
    circuit.append(cnot_64, qargs=explode(iv[2], s[3]))
    circuit.append(cnot_64, qargs=explode(key[1], s[2]))
    circuit.append(cnot_64, qargs=explode(iv[1], s[2]))
    circuit.append(cnot_64, qargs=explode(key[2], s[1]))
    circuit.append(cnot_64, qargs=explode(key[3], s[0]))
    circuit.append(cnot_64, qargs=explode(iv[3], s[0]))

    for _ in range(32):
        circuit.append(strumok_next, qargs=explode(s, r_1, r_2, tmp, carry))
        circuit.reset(carry[0])
        circuit.append(reset_64, qargs=explode(tmp))

    return circuit


In [143]:
def strumok_stream_circuit() -> QuantumCircuit:
    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    carry = QuantumRegister(1, 'carry')

    # Stream output
    out = QuantumRegister(64, 'out')

    # Circuit Input
    regs = []
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(carry)
    regs.append(out)

    circuit = QuantumCircuit(*regs, name='strumok_next')
    circuit.append(inplace_xor_circuit(64), qargs=explode(s[0], out))
    circuit.append(strumok_fsm_circuit(), qargs=explode(s[15], r_1, r_2, out, carry))

    return circuit